# ***Installing Dependencies:***

In [1]:
pip install google-api-python-client

In [2]:
pip install transformers[sentencepiece] --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.9 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')
classifier.model.name_or_path

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

'distilbert-base-uncased-finetuned-sst-2-english'

In [4]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Your API key here
api_key = "Please use own API Key."

# YouTube video ID
video_id = "RTXS4MMngnA"

# Number of comments to retrieve per request (max 100)
max_results = 50

# Number of likes constraint
max_likes = 2000

def get_video_details(video_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)

    try:
        response = youtube.videos().list(
            part="snippet",
            id=video_id
        ).execute()

        title = response["items"][0]["snippet"]["title"]
        tags = response["items"][0]["snippet"]["tags"] if "tags" in response["items"][0]["snippet"] else []

        return title, tags

    except HttpError as e:
        print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
        return None, None

video_title, video_tags = get_video_details(video_id, api_key)

if video_title and video_tags:
    print(f"Video Title: {video_title}")
    print(f"Tags: {', '.join(video_tags)}")
else:
    print("Unable to fetch video details.")


Video Title: Minecraft Speedrunner VS 3 Hunters REMATCH
Tags: Dream Minecraft, dream Minecraft youtube, minecraft, dream, Speedrunner vs 3 hunters, Speedrunner vs someone trying to stop them, Beating Minecraft but my friend tries to stop me, Minecraft but water rises every minute, minecraft but lava rises every minute, minecraft but challenge, minecraft challenge, challenge, minecraft 1.14, beating minecraft, rematch


In [5]:
classifier("I still can’t get over that strength potion bait.")

[{'label': 'NEGATIVE', 'score': 0.997313916683197}]

In [6]:
def get_video_comments(video_id, api_key, max_results):
    youtube = build("youtube", "v3", developerKey=api_key)

    comments = []
    nextPageToken = None

    while True:
        try:
            response = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=max_results,
                pageToken=nextPageToken
            ).execute()

            for item in response["items"]:
                comment = item["snippet"]["topLevelComment"]["snippet"]
                like_count = comment["likeCount"]
                if like_count >= max_likes:
                    comments.append(comment["textDisplay"])

            nextPageToken = response.get("nextPageToken")

            if not nextPageToken:
                break

        except HttpError as e:
            print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
            break

    return comments

video_comments = get_video_comments(video_id, api_key, max_results)
print("Extracted YouTube comments completed.")

Extracted YouTube comments completed.


In [7]:
import regex as re
#using regex to remove unnecessary tokens or phrases from text
def clean_text(text):
    #remove URLs.
    text = re.sub(r'http\S+', ' ', text)
    #remove HTML tags.
    text = re.sub(r'<.*?>|quot', ' ', text)
    #remove special characters and punctuation.
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    #remove extra whitespace.
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

positive_count = 0
negative_count = 0
count = 0

for i, comment in enumerate(video_comments, start=1):
    comment = clean_text(comment)
    print(f"Comment {i}: {comment}")
    classified = classifier(comment)
    print(classified)
    print("\n")

    if (classified[0]["label"] == "POSITIVE"):
        positive_count += 1
    else:
        negative_count += 1

    count = i

Comment 1: SUBSCRIBE sdhjdjhsdhjsdjhakjasjasksak
[{'label': 'NEGATIVE', 'score': 0.9782623052597046}]


Comment 2: Dream with his whole entire inventory Thats a surprise tool thatll help us later
[{'label': 'POSITIVE', 'score': 0.9980422258377075}]


Comment 3: He has so much food Little does sapnap know dream only has a wooden sword and his will to live
[{'label': 'NEGATIVE', 'score': 0.9959413409233093}]


Comment 4: The best part is when Dream collects a variable of random resources as we try to figure out or remember what hes planning
[{'label': 'POSITIVE', 'score': 0.9839948415756226}]


Comment 5: Dream aggressively drops shovel
[{'label': 'NEGATIVE', 'score': 0.9868820309638977}]


Comment 6: Dream on full get away from meeeee Dream on half a heart activates killer mode
[{'label': 'POSITIVE', 'score': 0.8720918893814087}]


Comment 7: Dream with full hearts stay away from me Dream with hearts cmere Sapnap
[{'label': 'POSITIVE', 'score': 0.9916158318519592}]


Comment 8: I SEE HI

In [9]:
print(f"Number of comments with {max_likes} likes: {count}")
print("Number of POSITIVE Comments:", positive_count)
print("Number of NEGATIVE Comments:", negative_count,"\n")

positivity = round(positive_count/(positive_count + negative_count) * 100)
negativity = round(negative_count/(negative_count + positive_count) * 100)

print(f"Positive Percentage: {positivity}%")
print(f"Negative Percentage: {negativity}%\n")

if (positivity > negativity):
    print("Final Deduction: The comments are more positive.")
else:
    print("Final Deduction: The comments are more negative.")

Number of comments with 2000 likes: 177
Number of POSITIVE Comments: 68
Number of NEGATIVE Comments: 109 

Positive Percentage: 38%
Negative Percentage: 62%

Final Deduction: The comments are more negative.
